In [1]:
import pandas as pd
import time


from detector_radford import DetectorRadford
from detector_detectgpt import DetectorDetectGPT
from detector_guo import DetectorGuo
from detector_dummy import DetectorDummy
from explainer_wrappers import Anchor_Explainer,LIME_Explainer, SHAP_Explainer, Anchor_Explainer
detector_classes = [DetectorGuo, DetectorRadford]
explainer_classes = [ SHAP_Explainer,LIME_Explainer]
results = []


In [2]:
import transformers
print(transformers.__version__)

4.25.1


In [3]:
test = pd.read_pickle("./dataset_test.pkl")
train = pd.read_pickle("./dataset_train.pkl")

In [4]:
len(test)+len(train)

1016

In [5]:
len(test)

305

In [6]:
3*3*2*2

36

In [7]:
documents = train["answer"]
gold_labels = train["author"] == "human_answers"

In [8]:
DetectorRadford().predict_label([documents[0]])

array([1], dtype=int64)

In [9]:
76iutzjgh

SyntaxError: invalid decimal literal (1911369227.py, line 1)

In [ ]:
from tqdm import tqdm

In [ ]:
# for detector_class in detector_classes:
#     detector = detector_class()
#     start = time.time()
#     y = detector.predict_label(documents)
#     end = time.time()

#     acc = sum(y == gold_labels)/ len(documents)
#     results.append((detector.__class__.__name__, acc, end - start))
#     print(results[-1])


In [ ]:
# detector = DetectorDetectGPT()

In [ ]:
# detector.get_pad_token_id_masker()

In [ ]:
import numpy as np

In [ ]:
# y = detector.predict_proba(documents, deterministic=True)
# yy = np.array(y).argmax(axis=1)
# acc = sum(yy == gold_labels)/ len(documents)
# acc

In [ ]:
# y = detector.predict_proba(documents, deterministic=True)
# yy = np.array(y).argmax(axis=1)
# acc = sum(yy == gold_labels)/ len(documents)
# acc

In [ ]:
from IPython.core.display import display, HTML

Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display


In [ ]:
# for document, gt in zip(documents,gold_labels):
#     explainer = SHAP_Explainer(detector)
#     print(gt)
#     display(HTML(explainer.get_vanilla_visualization_HTML(document)))

In [ ]:
# for document in tqdm(documents):
#     detector.predict_proba([document])

In [ ]:
import shap

In [ ]:
for detector_class in detector_classes:
    detector = detector_class()
    for explainer_class in explainer_classes:
        explainer = explainer_class(detector)
        print(explainer.__class__.__name__, detector.__class__.__name__)
        docs = [document for document in documents if not explainer.is_cached(document)]
        #docs = docs[0:len(docs)//2]
        for document in tqdm(docs):
            explainer.get_explanation_cached(document)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


SHAP_Explainer DetectorGuo


100%|██████████| 711/711 [1:49:26<00:00,  9.24s/it]


LIME_Explainer DetectorGuo


100%|██████████| 711/711 [3:39:30<00:00, 18.52s/it]  


SHAP_Explainer DetectorRadford


100%|██████████| 711/711 [1:52:49<00:00,  9.52s/it]


LIME_Explainer DetectorRadford


100%|██████████| 711/711 [4:06:20<00:00, 20.79s/it]  


In [ ]:
# detector = DetectorDetectGPT()


In [ ]:
345wter78/2

SyntaxError: invalid decimal literal (4294918467.py, line 1)

In [ ]:
# # detector = DetectorRadford()
# explainer = LIME_Explainer(detector)

# explainer.get_fi_scores_batch(documents)

In [ ]:
# explainer = LIME_Explainer(detector)

# explainer.get_fi_scores_batch(documents)

In [ ]:
# detector=DetectorDetectGPT()

In [ ]:
# explainer = Anchor_Explainer(detector)


In [ ]:
exp = None

In [ ]:
# sent = "This is a sentence, machine human."
# sent

In [ ]:
gold_labels.iloc[3]

False

In [ ]:
# words, positions, true_label, sample_fn = explainer.explainer.get_sample_fn(sent, detector.predict_label, onepass=True)

In [ ]:
# sentences = sample_fn([], 100)
# # ["".join(a) for a in sentences.reshape(sentences.shape[0], -1)]

In [ ]:
# sentences = sample_fn([], 100)
# # ["".join(a) for a in sentences.reshape(sentences.shape[0], -1)]

In [ ]:
# 5ztrhdgf

In [ ]:
import sys

In [ ]:
text = 'An example is a specific instance or illustration that represents a general idea or concept. It serves to clarify or demonstrate a point, making complex ideas more understandable. For instance, a red apple can be an example of a fruit, showcasing the broader category of fruits through a specific case.'

In [ ]:
i = 3
text = documents.iloc[i]
print(gold_labels.iloc[i])
text

False


'Institutional investors are organizations that invest on behalf of their clients, such as pension funds, mutual funds, endowments, and insurance companies. These investors typically have a large amount of capital to invest and may be seeking to diversify their portfolios across a variety of asset classes. Institutional investors may also be seeking to generate returns or achieve specific investment goals on behalf of their clients. They may conduct extensive research and analysis to identify investment opportunities and make informed decisions about where to allocate their capital. Institutional investors play a significant role in financial markets and often have a significant impact on the prices of securities and other assets.'

In [ ]:
gold_labels.iloc[i]

False

In [ ]:
import random

In [ ]:
texts = [" ".join([t if random.random() < 0.95 else detector.get_pad_token() for t in text.split(" ")]) for _ in range(0,100)]

In [ ]:
detector.predict_proba([text])

array([[1, 0]])

In [ ]:
detector.predict_label([text])

In [ ]:
# # ChatGPT prompt: "What is an example? Answer in 50 words"
# exp = explainer.get_explanation_cached('An example is a specific instance or illustration that represents a general idea or concept. It serves to clarify or demonstrate a point, making complex ideas more understandable. For instance, a red apple can be an example of a fruit, showcasing the broader category of fruits through a specific case.')

In [ ]:
from anchor.anchor import anchor_explanation

In [ ]:
# anchor_explanation.AnchorExplanation('text', exp, explainer.explainer.as_html).show_in_notebook()

In [ ]:
# isinstance(explainer, Anchor_Explainer)

In [ ]:
computation_times = []

In [ ]:
for document, gt  in tqdm(zip(documents, gold_labels)):

    exp = explainer.get_explanation_cached(document)
    if isinstance(explainer, Anchor_Explainer):
        computation_times.append((exp["computation_time"], gt, len(document.split(" "))))
   # anchor_explanation.AnchorExplanation('text', exp, explainer.explainer.as_html).show_in_notebook()
    df = pd.DataFrame(computation_times, columns=["time", "gt",  "len"])
    display(df)
    print(df["time"].mean())


In [ ]:
pd.DataFrame(computation_times, columns=["time", "gt", "prediction", "len"])